<a href="https://colab.research.google.com/github/Avenge-PRC777/RAG/blob/main/Retrieval_Augmented_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Introduction

In [1]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install chromadb
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.8 MB/s eta

In [3]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import openai
import os

In [6]:
os.environ["OPENAI_API_KEY"] = open("openapi_key.txt", "r", encoding = "utf-8").read().strip()

In [4]:
#loader = TextLoader("./tax_custom_rules_2.tsv")
#loader = TextLoader("./tax_custom_rules.tsv")
loader = PyPDFLoader("finance_bill_2023.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [7]:
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())

In [9]:
query = "My income is 2.3 lakhs, how much tax do I pay"
qa.run(query)

' You pay Rs.10,000 plus 20 per cent. of the amount by which the total income exceeds Rs. 5,00,000, which is Rs. 10,000 plus 20 per cent. of the amount by which the total income exceeds Rs. 5,00,000, so you would pay Rs. 12,000 in total.'

In [18]:
query = "My income is 5.3 lakhs, how much tax do I pay"
qa.run(query)

' 10%, as you have an income of more than 5 lakhs.'

In [10]:
query = "My income is 5.3 lakhs, how much tax do I pay and I am widowed"
qa.run(query)

' You pay Rs. 10,000 plus 20% of the amount by which the total income exceeds Rs. 5,00,000, which would be Rs. 10,000 + (20% x Rs. 30,000) = Rs. 16,000.'

In [17]:
query = "What are deductions available under the Income Tax Act."
qa.run(query)

' This context does not provide information about deductions available under the Income Tax Act.'